In [4]:
# !pip3 install torch
# !pip3 install numpy

import torch
import random
import numpy as np
from torch.utils.data import dataloader, dataset
import torchvision
#
# random.seed(0)
# np.random.seed(0)
# torch.manual_seed(0)
# torch.cuda.manual_seed(0)
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.models.detection import transform
from torchvision import transforms
torch.backends.cudnn.deterministic = True

In [6]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [1]:
import csv
import os
from re import search

def make_img_folder(dataPath, csvPath, imgFolder):
    with open(csvPath) as f:
        reader = csv.reader(f)
        for row in reader:
            if row[0].isdigit():
                print(f"cp {dataPath}/{'.'.join([row[0],'png'])} {imgFolder}/{row[1]}/")
                os.system(f"mv {dataPath}/{'.'.join([row[0],'png'])} {imgFolder}/{row[1]}/")
Data_path = "/home/alexsun/diskb/streamlabs/task/images"
csv_path = "/home/alexsun/diskb/streamlabs/task/images_labelling.csv"
imgFolder = "/home/alexsun/diskb/streamlabs/task/data"
# make_img_folder(Data_path, csv_path, imgFolder)

In [8]:
dataset = torchvision.datasets.ImageFolder(imgFolder,transform = transforms.ToTensor())
batch_size = 16
validation_split = .3
shuffle_dataset = True
random_seed= 8

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset,
                                                sampler=valid_sampler)
# dataset.shape

In [29]:
from torchvision import datasets, transforms as T
import torch

model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
model.eval()

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

for epoch in range(10000):
    for i, btch in enumerate(train_loader):
        X_train, labels_train = btch
        x_train = torch.Tensor(X_train).float()
        optimizer.zero_grad()

        preds = model.forward(X_train)

        loss_value = loss(preds, labels_train)
        loss_value.backward()

        optimizer.step()

    # acc = []
    x_test, target = validation_loader
    test_preds = model.forward(x_test)
    # test_loss_history.append(loss(test_preds, target))
    acc=(test_preds.argmax(dim=1) == target).float()
    # train_preds = mnist_net.forward(X_train)
    # train_loss_history.append(loss(train_preds, y_train))

    # print(acc)
    accuracy =acc.mean()
    # accuracy = (test_preds.argmax(dim=1) == y_test).float().mean()
    # accuracy = (train_preds.argmax(dim=1) == y_train).float().mean()
    # test_accuracy_history.append(accuracy)
    # train_accuracy_history.append(accuracy)
    print(accuracy)


Using cache found in /home/alexsun/.cache/torch/hub/pytorch_vision_v0.6.0


RuntimeError: stack expects each tensor to be equal size, but got [3, 84, 46] at entry 0 and [3, 70, 42] at entry 1